In [1]:
import config
from pathlib import Path

from requests import request
import zipfile, io, os, pandas as pd

from time import sleep

# Setting user & survey parameters
Assumes user has api token generated and stored in text file at `fpath`. Change this path and encoding method in the cell below to whatever your preferred method is.

List as many or as few qualtrics survey IDs as needed in `0.1.qualtrics-request-list.txt`. Running the request cell later may produce errors if you're exporting multiple surveys due to time restrictions on the API request. The code tries to deal with this in a relatively haphazard way by just waiting a couple seconds between requests, which works *almost* all the time. If you **do** run into errors along that line, re-running the cell will usually fix it... eventually.

In [2]:
debug = False

In [3]:
apiToken = config.api_token
surveyIdList = config.survey_list
fileFormat = "csv"
dataCenter = 'ca1'
output_dir = Path('..') / '..' / 'sourcedata' / 'qualtrics'

# Setting static parameters

In [4]:
requestCheckProgress = 0
progressStatus = "in progress"
baseUrl = "https://{0}.qualtrics.com/API/v3/responseexports/".format(dataCenter)
headers = {
    "content-type": "application/json",
    "x-api-token": apiToken,
    }

# Executing Request
If executing in Jupyter, this is the cell that will throw an error for each survey.

Simply re-execute until it stops throwing errors...

In [5]:
i=1
for surveyId in surveyIdList:
    print('survey {} of {}'.format(i,len(surveyIdList)));i+=1
    print('surveyId',surveyId)
    
    # Step 1: Creating Data Export
    downloadRequestUrl = baseUrl
    downloadRequestPayload = '{"format":"' + fileFormat + '","surveyId":"' + surveyId + '"}'
    downloadRequestResponse = request("POST", downloadRequestUrl, data=downloadRequestPayload, headers=headers)
    progressId = downloadRequestResponse.json()["result"]["id"]
    
    if debug == True:
        print('downloadRequestUrl',downloadRequestUrl)
        print('downloadRequestPayload',downloadRequestPayload)
        print('downloadRequestResponse',downloadRequestResponse)
        print('progressId',progressId)
    
    # Step 2: Checking on Data Export Progress and waiting until export is ready
    while requestCheckProgress < 99 and progressStatus is not "complete":
        requestCheckUrl = baseUrl + progressId
        requestCheckResponse = request("GET", requestCheckUrl, headers=headers)
        requestCheckProgress = requestCheckResponse.json()["result"]["percentComplete"]
        print("Download is " + str(requestCheckProgress) + "% complete")
    print("Download is " + str(requestCheckProgress) + "% complete")
    
    if debug == True:
        print('requestCheckUrl',requestCheckUrl)
        print('requestCheckResponse',requestCheckResponse)
        print('requestCheckProgress',requestCheckProgress)
        
    # Step 3: Downloading file
    requestDownloadUrl = baseUrl + progressId + '/file'
    sleep(2)
    requestDownload = request("GET", requestDownloadUrl, headers=headers, stream=True)
        
    if debug == True:
        print('requestDownloadUrl',requestDownloadUrl)
        print('requestDownload',requestDownload)
    
    # Step 4: Unzipping the file
    zipfile.ZipFile(io.BytesIO(requestDownload.content)).extractall(output_dir)
print('Done!')

survey 1 of 4
surveyId SV_bsl9vF012E6Thbv
Download is 100.0% complete
Download is 100.0% complete
survey 2 of 4
surveyId SV_bBNxUVhoGFcyKot
Download is 100.0% complete
survey 3 of 4
surveyId SV_2gAX1ty1UjOeKBT
Download is 100.0% complete
survey 4 of 4
surveyId SV_d4o0ZLBDB8GxpwV
Download is 100.0% complete
Done!


# Read outputs, clean and re-write

In [6]:
columns_to_drop=[
    'StartDate','EndDate','Status','IPAddress','Finished',
    'RecipientLastName','RecipientFirstName','RecipientEmail',
    'LocationLatitude','LocationLongitude','LocationAccuracy',
    'ResponseID','ResponseSet','ExternalDataReference',
    'Score-weightedAvg','Score-weightedStdDev'
]

In [7]:
for f in os.listdir(output_dir):
    fpath = os.path.join(output_dir,f)
    if os.path.isfile(fpath):
        print('backing up {}'.format(fpath))
        df = pd.read_csv(fpath)
        df.to_csv(os.path.join(output_dir,'raw',os.path.basename(fpath)),index=False)
        print('done')
        print('cleaning {}'.format(fpath))
        df = df[df['Finished'] == '1']
        df = df[[
            c for c in df.columns
            if not c.startswith('DO-')
            and not c.startswith('RO-')
            and not c in columns_to_drop
        ]]
        df.to_csv(fpath,index=False)
        print('done')

backing up ..\..\sourcedata\qualtrics\Comprehension-1 FullSample.csv
done
cleaning ..\..\sourcedata\qualtrics\Comprehension-1 FullSample.csv
done
backing up ..\..\sourcedata\qualtrics\Comprehension-2 FullSample.csv
done
cleaning ..\..\sourcedata\qualtrics\Comprehension-2 FullSample.csv
done
backing up ..\..\sourcedata\qualtrics\Survey-01 FullSample.csv
done
cleaning ..\..\sourcedata\qualtrics\Survey-01 FullSample.csv
done
backing up ..\..\sourcedata\qualtrics\Survey-02 FullSample.csv
done
cleaning ..\..\sourcedata\qualtrics\Survey-02 FullSample.csv
done


# Next Step
## Data cleaning & combination
Now that we have collected the qualtrics data and done preliminary cleaning, open the next workbook: [0.2.clean_qualtrics.ipynb](0.2.clean_qualtrics.ipynb).

This workbook will combine the pre- and post- surveys, as well as compute our pre-test and post-test comprehension to get our subject level outcome measures.